In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_naver_news(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.find('h2', {'id': 'title_area'})
        title_text = title.text.strip() if title else "제목 없음"
        content = soup.find('article', {'id': 'dic_area'})
        summary = soup.find('strong', {'class': 'media_end_summary'})
        datetime = soup.find('span', {'class': "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"})
        summary_text = summary.text.strip() if summary else "요약 없음"
        datetime_text = datetime.text.strip().split(' ')[0] if datetime else "날짜 없음"
        if content:
            for script in content(['script', 'style']):
                script.decompose()
            text = "".join(content.find_all(string=True, recursive=False)).strip()
            df = pd.DataFrame({'title': [title_text], 'datetime': [datetime_text], 'summary': [summary_text], 'content': [text], 'url': [url]})
            return df
        else:
            return pd.DataFrame({'content': ["본문 없음"]})
    else: 
        return pd.DataFrame({'content': ["페이지 없음"]})

In [3]:
url = "https://n.news.naver.com/mnews/article/014/0005289562?sid=105"
content = get_naver_news(url)
print(content)

                 title     datetime  \
0  셀트리온 "CDMO로 제2의 도약"  2024.12.31.   

                                             summary  \
0  자회사 바이오솔루션스 공식출범국내 20만L 규모 생산시설 구축2031년 연매출 3조...   

                                             content  \
0  바이오시밀러의 글로벌 강자인 셀트리온이 바이오의약품 위탁개발생산(CDMO) 시장에 ...   

                                                 url  
0  https://n.news.naver.com/mnews/article/014/000...  


In [ ]:
import time
import random
import os

file_path = r"c:\\Programming\\소캡디\\셀트리온_news_links_1.txt"
print(f"파일 경로: {file_path}")
print(f"파일 존재 여부: {os.path.exists(file_path)}")

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        urls = [line.strip() for line in file if line.strip()]
    
    print(f"읽어온 URL 수: {len(urls)}")
    if len(urls) > 0:
        print(f"첫 번째 URL: {urls[0]}")
    else:
        print("URL을 읽어오지 못했습니다. 파일 내용을 직접 확인해보세요.")
        
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                print(f"파일 내용 일부: {content[:200]}...")
                print(f"파일 길이: {len(content)} 문자")
        except Exception as file_e:
            print(f"파일 내용 확인 중 오류: {str(file_e)}")

    if urls:
        print(f"총 {len(urls)}개의 URL을 처리합니다.")
        
        all_data = []
        
        for i, url in enumerate(urls):
            try:
                print(f"URL 처리 중 {i+1}/{len(urls)}: {url}")
                df = get_naver_news(url)
                if not df.empty and 'content' in df.columns and df['content'].iloc[0] not in ["본문 없음", "페이지 없음"]:
                    all_data.append(df)
                else:
                    print(f"  URL에서 컨텐츠를 가져오지 못했습니다: {url}")
            except Exception as e:
                print(f"  URL 처리 중 오류 발생: {url}: {str(e)}")
            
            # 서버 부하를 줄이기 위한 지연 시간 추가
            if i < len(urls) - 1:
                delay = random.uniform(0.5, 2.0)
                time.sleep(delay)
        
        if all_data:
            combined_df = pd.concat(all_data, ignore_index=True)
            
            output_path = r"c:\\Programming\\소캡디\\naver_news_data.csv"
            combined_df.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f"{len(combined_df)}개의 뉴스 기사를 {output_path}에 저장했습니다.")
            
            # 결과 데이터 확인
            print("\n수집된 데이터의 처음 몇 행:")
            display(combined_df.head())
        else:
            print("수집된 데이터가 없습니다.")
    else:
        print("처리할 URL이 없습니다.")
        
except Exception as e:
    print(f"파일 읽기 중 오류 발생: {str(e)}")

파일 경로: c:\\Programming\\소캡디\\셀트리온 공시자료\\news.txt
파일 존재 여부: True
읽어온 URL 수: 100
첫 번째 URL: https://n.news.naver.com/mnews/article/014/0005289562?sid=105
총 100개의 URL을 처리합니다.
URL 처리 중 1/100: https://n.news.naver.com/mnews/article/014/0005289562?sid=105
URL 처리 중 2/100: https://n.news.naver.com/mnews/article/016/0002409174?sid=101
URL 처리 중 3/100: https://n.news.naver.com/mnews/article/018/0005915333?sid=101
URL 처리 중 4/100: https://n.news.naver.com/mnews/article/648/0000031912?sid=101
URL 처리 중 5/100: https://n.news.naver.com/mnews/article/009/0005420505?sid=105
URL 처리 중 6/100: https://n.news.naver.com/mnews/article/029/0002925776?sid=105
URL 처리 중 7/100: https://n.news.naver.com/mnews/article/030/0003271658?sid=102
URL 처리 중 8/100: https://n.news.naver.com/mnews/article/011/0004432275?sid=101
URL 처리 중 9/100: https://n.news.naver.com/mnews/article/011/0004431592?sid=101
URL 처리 중 10/100: https://n.news.naver.com/mnews/article/015/0005073658?sid=105
URL 처리 중 11/100: https://n.news.naver.com/mnews/

,title,datetime,summary,content,url
0,"셀트리온 ""CDMO로 제2의 도약""",2024.12.31.,자회사 바이오솔루션스 공식출범국내 20만L 규모 생산시설 구축2031년 연매출 3조...,바이오시밀러의 글로벌 강자인 셀트리온이 바이오의약품 위탁개발생산(CDMO) 시장에 ...,https://n.news.naver.com/mnews/article/014/000...
1,셀트리온 ‘다잘렉스’ 시밀러 3상 美 FDA 승인,2024.12.30.,요약 없음,셀트리온이 미국 식품의약국(FDA)으로부터 다잘렉스(성분명 다라투무맙)의 바이오시밀...,https://n.news.naver.com/mnews/article/016/000...
2,"[특징주]셀트리온, 1000억 자사주 추가 취득 결정에 3%↑",2024.12.30.,요약 없음,[이데일리 김응태 기자] 셀트리온이 1000억원 규모의 자사주 취득을 결정하면서 강...,https://n.news.naver.com/mnews/article/018/000...
3,"셀트리온, 자사주 54만주 추가 취득",2024.12.27.,"""기업·주주 가치 제고""",셀트리온은 오는 30일부터 장내매수를 통해 자사주 54만6747주를 취득한다고 27...,https://n.news.naver.com/mnews/article/648/000...
4,"셀트리온, 자사주 1000억 추가 취득",2024.12.27.,요약 없음,셀트리온이 약 1000억원(26일 종가 기준) 규모의 자사주 추가 매입을 결정했다고...,https://n.news.naver.com/mnews/article/009/000...
